In [36]:
from lxml import html, etree
import urllib2
import requests
import HTMLParser

url = 'http://18.fsin.su/uchrezdeniya/msch-18/'
page = requests.get('http://18.fsin.su/uchrezdeniya/msch-18/')
tree = html.document_fromstring(page.content)
aa = tree.xpath("//div[@class='l-content-holder']")[0]
print HTMLParser.HTMLParser().unescape(html.tostring(aa))

<div class="l-content-holder">
                                <div class="b-content margin">
                                    <div class="bb-title"><h1 class="h1">Федеральное казеное учреждение здравоохранения "Медико-Санитарная Часть № 18 ФСИН России"</h1></div>
                                    <div class="separator"></div>
                         
<p class="MsoNormal"><font size="2" face="Arial, sans-serif" color="#333333"><b>Почтовый адрес: </b></font></p>
 
<p class="MsoNormal"><span style="color: rgb(51, 51, 51); font-family: Arial, sans-serif; font-size: small; ">426004, г. Ижевск, ул. Коммунаров, 216</span></p>
 
<p class="MsoNormal"><b style="color: rgb(51, 51, 51); font-family: Arial, sans-serif; font-size: small; ">Контактные телефоны: </b></p>
 
<p class="MsoNormal"><span style="color: rgb(51, 51, 51); font-family: Arial, sans-serif; font-size: small; ">Приемная: 8 (3412) 631-937</span></p>
 
<p class="MsoNormal"><span style="color: rgb(51, 51, 51); font-family: Aria

In [ ]:
import HTMLParser
url = 'http://18.fsin.su/uchrezdeniya/msch-18/'
page = requests.get('http://18.fsin.su/uchrezdeniya/msch-18/')
HTMLParser.HTMLParser().unescape(page.content)

In [ ]:
from BeautifulSoup import BeautifulStoneSoup

In [ ]:
import SimpleHTTPServer
import SocketServer

class MyRequestHandler(SimpleHTTPServer.SimpleHTTPRequestHandler):
    def do_GET(self):
        #if self.path == '/':
        self.path = '1.html'
        return SimpleHTTPServer.SimpleHTTPRequestHandler.do_GET(self)

Handler = MyRequestHandler
server = SocketServer.TCPServer(('', 8000), Handler)

server.serve_forever()

In [106]:
import sqlite3 as lite
import sys

try:
    con = lite.connect('test.db')

    cur = con.cursor()  

    cur.executescript("""
        CREATE TABLE IF NOT EXISTS Cars(Id INT, Name TEXT, Price INT);
        INSERT INTO Cars VALUES(1,'Audi',52642);
        INSERT INTO Cars VALUES(2,'Mercedes',57127);
        INSERT INTO Cars VALUES(3,'Skoda',9000);
        INSERT INTO Cars VALUES(4,'VolvoАААббб',29000);
        INSERT INTO Cars VALUES(5,'Bentley',350000);
        INSERT INTO Cars VALUES(6,'Citroen',21000);
        INSERT INTO Cars VALUES(7,'Hummer',41400);
        INSERT INTO Cars VALUES(8,'Volkswagen',21600);
        """)

    con.commit()
    cur.execute("SELECT * FROM Cars")

    rows = cur.fetchall()

    for row in rows:
        print row[1]
    
except lite.Error, e:
    
    if con:
        con.rollback()
        
    print "Error %s:" % e.args[0]
    sys.exit(1)
    
finally:
    
    if con:
        con.close() 

Audi
Mercedes
Skoda
VolvoАААббб
Bentley
Citroen
Hummer
Volkswagen
Audi
Mercedes
Skoda
VolvoАААббб
Bentley
Citroen
Hummer
Volkswagen
Audi
Mercedes
Skoda
VolvoАААббб
Bentley
Citroen
Hummer
Volkswagen
Audi
Mercedes
Skoda
VolvoАААббб
Bentley
Citroen
Hummer
Volkswagen
Audi
Mercedes
Skoda
VolvoАААббб
Bentley
Citroen
Hummer
Volkswagen


In [2]:
import flask
import simplejson as json
import sqlite3 as lite
import requests
from lxml import html
from lxml.html.diff import htmldiff
import HTMLParser
from datetime import datetime

class WebApplication:
    def __init__(self):
        self.app = flask.Flask(__name__, static_url_path = '')

        @self.app.route('/js/<path:path>')
        def send_js(path):
            return flask.send_from_directory('js', path)

        @self.app.route('/', methods = ['GET', 'POST'])
        def login():
            if flask.request.method == 'POST':
                print flask.request.form['name']
                return 'OK'
            else:
                print 234
                return self.app.send_static_file('index.html')
    
    def run(self):
        self.app.run()

class FsinParser:
    def __init__(self):
        self.json_name = '1.json'
        with open(self.json_name, 'r') as f:
            self.json = json.load(f)
            f.close()
        self.db = lite.connect(self.json['db_name'])
        self.db.text_factory = str
        self.cursor = self.db.cursor()
        self.pages_list = []
        try:
            self.cursor.executescript("CREATE TABLE IF NOT EXISTS \
                                      PAGES_HISTORY(Id INT, Info STRING, Datetime DATETIME);")
            self.cursor.executescript("CREATE TABLE IF NOT EXISTS \
                                      PAGES_LIST(Id INT, Url STRING);")
        except:
            print '__init__: SQL Error'
        self.update_pages_list()
        
    def update_pages_list(self):
        try:
            self.cursor.execute("SELECT * FROM PAGES_LIST")
            self.pages_list = self.cursor.fetchall()
        except:
            print ': SQL Error'
    
    def get_page_from_internet(self, url):
        try:
            page = requests.get(url)
            tree = html.document_fromstring(page.content)
            info = tree.xpath(self.json['xpath_request'])[0]
            return HTMLParser.HTMLParser().unescape(html.tostring(info)).encode('utf-8')
        except:
            print ': SQL Error'
    
    def update_pages_history_by_index(self, index):
        try:
            page_id = self.pages_list[index][0]
            new_page = self.get_page_from_internet(self.pages_list[index][1])
            self.cursor.execute("SELECT * FROM PAGES_HISTORY WHERE Id=? ORDER BY datetime(Datetime) DESC LIMIT 1",
                                (page_id,))
            old_page_entry = self.cursor.fetchall()
            if len(old_page_entry) != 0:
                if old_page_entry[0][1] != new_page:
                    self.cursor.execute("INSERT INTO PAGES_HISTORY VALUES(?, ?, ?)", 
                                        (page_id, new_page, datetime.now()))
                    self.db.commit()
        except:
            print ': SQL Error'
    
    def update_pages_history(self):
        self.update_pages_list()
        for index in range(self.pages_list):
            update_pages_history_by_index(index)
            
    
    def put_url_to_pages_list(self, url):
        try:
            max_id = -1
            for page in self.pages_list:
                if (page[0] > max_id):
                    max_id = page[0]
            max_id += 1
            self.cursor.execute("INSERT INTO PAGES_LIST VALUES(?, ?)", (max_id, url))
            self.db.commit()
        except:
            print ': SQL Error'
    
    def get_diff_of_pages(self, page1, page2):
        diff = HTMLParser.HTMLParser().unescape(htmldiff(page1, page2).encode('utf-8'))
        diff = diff.replace('<del>', '<del style="color:red">')
        diff = diff.replace('<ins>', '<ins style="color:green">')
        return diff
    
    def finish(self):
        if (self.db):
            self.db.close()
        
    

if __name__ == '__main__':
    # app = WebApplication()
    # app.run()
    parser = FsinParser()
    parser.update_pages_history_by_index(1)
    #parser.put_url_to_pages_list('http://48.fsin.su/structure/ispravitelnaya-koloniya-2.php')
    #print parser.pages_list
    #with open('out1.txt', 'w') as f:
    #    f.write(parser.get_info_from_page(0))
    #    f.close()

In [8]:
#import difflib
from lxml.html.diff import htmldiff
#html_diff = difflib.HtmlDiff()

url = 'http://18.fsin.su/uchrezdeniya/msch-18/'
page = requests.get('http://48.fsin.su/structure/ispravitelnaya-koloniya-2.php')
tree = html.document_fromstring(page.content)
aa = tree.xpath("//div[@class='l-content-holder']")[0]
data =  HTMLParser.HTMLParser().unescape(html.tostring(aa)).encode('utf-8')
data1 = HTMLParser.HTMLParser().unescape(html.tostring(aa))

data1 = data1.replace('ФСИН'.decode('utf-8'), 'jopa')
#print ''.join(difflib.context_diff(data, data1))

diff = HTMLParser.HTMLParser().unescape(htmldiff(data, data1).encode('utf-8'))
print data
diff = diff.replace('<del>', '<del style="color:red">')
diff = diff.replace('<ins>', '<ins style="color:green">')
#print diff
with open('table.html', 'w') as f:
    f.write(HTMLParser.HTMLParser().unescape(diff))

<div class="l-content-holder">
                                <div class="b-content margin">
                                    <div class="bb-title"><h1 class="h1">Исправительная колония №2</h1></div>
                                    <div class="separator"></div>
                         
<p> ИК-2 <b>Федеральное казенное учреждение "Исправительная колония №2"</b> 
  <br>
 </p>
 398027, г. Липецк, р-н Цементного завода 
<br>
 Начальник - Акинин Александр Александрович 
<br>
 тел. <img width="0" height="0" src="resource://skype_ff_extension-at-jetpack/skype_ff_extension/data/call_skype_logo.png"><span class="skype_c2c_print_container notranslate">(4742) 48-37-19</span><span id="skype_c2c_container" class="skype_c2c_container notranslate" dir="ltr" tabindex="-1" onmouseover="SkypeClick2Call.MenuInjectionHandler.showMenu(this, event)" onmouseout="SkypeClick2Call.MenuInjectionHandler.hideMenu(this, event)" onclick="SkypeClick2Call.MenuInjectionHandler.makeCall(this, event)" data-numbe

In [74]:
from datetime import datetime
print datetime.now()

2015-12-03 12:21:12.000348
